# 🎨 บทที่ 2.4: Data Augmentation - เพิ่มข้อมูลภาพ

## 🎯 เรื่องราวของนักวาดภาพ

สมมติคุณมีรูปแมว **100 ภาพ** ต้องการเทรนโมเดลจำแนกแมว

**ปัญหา:**
- ข้อมูลน้อยเกินไป → โมเดลจะ **Overfit** (จำข้อมูลตายตัว)
- ไม่สามารถ generalize กับภาพใหม่ได้

**วิธีแก้:** เพิ่มข้อมูลด้วย **Data Augmentation**!

จาก 100 ภาพ → สร้างเป็น **1000+ ภาพ** โดย:
- หมุนภาพ 🔄
- ย่อ/ขยาย 🔍
- พลิกซ้าย-ขวา ↔️
- เลื่อนตำแหน่ง ⬆️⬇️
- ปรับความสว่าง ☀️🌙

**ผลลัพธ์:** โมเดลเห็นแมวในหลากหลายมุมมอง → **เรียนรู้ดีขึ้น!** ✨

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageFilter
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded!")

---

## 🖼️ Part 1: สร้างภาพตัวอย่าง

เราจะสร้างภาพ mock สี่เหลี่ยมสีต่างๆ เพื่อแสดงการ augment

In [ ]:
# สร้างภาพตัวอย่าง - รูปทรงเรขาคณิต
def create_sample_image(size=(224, 224)):
    """
    สร้างภาพตัวอย่างที่มีสี pattern ต่างๆ
    """
    img = Image.new('RGB', size, color='white')
    pixels = img.load()
    
    # สร้าง gradient และ shapes
    for i in range(size[0]):
        for j in range(size[1]):
            # สี gradient
            r = int((i / size[0]) * 255)
            g = int((j / size[1]) * 255)
            b = int(((i + j) / (size[0] + size[1])) * 255)
            pixels[i, j] = (r, g, b)
    
    # วงกลมตรงกลาง
    from PIL import ImageDraw
    draw = ImageDraw.Draw(img)
    center_x, center_y = size[0] // 2, size[1] // 2
    radius = 50
    draw.ellipse(
        [(center_x - radius, center_y - radius),
         (center_x + radius, center_y + radius)],
        fill='yellow',
        outline='red',
        width=3
    )
    
    # สี่เหลี่ยม
    draw.rectangle(
        [(50, 50), (100, 100)],
        fill='blue',
        outline='black',
        width=2
    )
    
    # ข้อความ
    from PIL import ImageFont
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()
    draw.text((10, 200), "SAMPLE IMAGE", fill='black', font=font)
    
    return img

# สร้างและบันทึกภาพ
os.makedirs('augmentation_demo', exist_ok=True)
original_img = create_sample_image()
original_img.save('augmentation_demo/original.jpg')

# แสดงภาพ
plt.figure(figsize=(6, 6))
plt.imshow(original_img)
plt.title('Original Image', fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()

print("✅ Sample image created!")

---

## 🔄 Part 2: Rotation (การหมุน)

### ทำไมต้องหมุน?
- วัตถุอาจอยู่ในมุมต่างๆ
- ภาพอาจเอียง

### มาดูผลของการหมุนในมุมต่างๆ

In [ ]:
# ทดลองหมุนในมุมต่างๆ
rotation_angles = [0, 45, 90, 135, 180, 270]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, angle in enumerate(rotation_angles):
    rotated = original_img.rotate(angle, expand=True)
    
    axes[idx].imshow(rotated)
    axes[idx].set_title(f'Rotation: {angle}°', fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("💡 สังเกต:")
print("   - ภาพหมุนแต่ยังเป็นภาพเดียวกัน")
print("   - โมเดลจะเรียนรู้ว่าวัตถุในมุมต่างๆ คือสิ่งเดียวกัน")
print("   - ใช้ rotation_range=20 หรือ 30 ดีที่สุด (ไม่มากเกินไป)")

---

## ↔️ Part 3: Flipping (การพลิก)

### 2 ประเภท:
1. **Horizontal Flip** (ซ้าย-ขวา) - ใช้บ่อย
2. **Vertical Flip** (บน-ล่าง) - ใช้น้อย

In [ ]:
from PIL import ImageOps

# สร้างภาพพลิก
horizontal_flip = ImageOps.mirror(original_img)
vertical_flip = ImageOps.flip(original_img)

# แสดงภาพ
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(original_img)
axes[0].set_title('Original', fontsize=13, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(horizontal_flip)
axes[1].set_title('Horizontal Flip ↔️\n(Mirror)', fontsize=13, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(vertical_flip)
axes[2].set_title('Vertical Flip ↕️\n(Upside Down)', fontsize=13, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("💡 Horizontal Flip:")
print("   ✅ ใช้กับ: สัตว์, วัตถุทั่วไป, ใบหน้า")
print("   ❌ ไม่ควรใช้กับ: ตัวเลข (6 กลับด้านเป็น 9), ตัวอักษร")

print("\n💡 Vertical Flip:")
print("   ✅ ใช้กับ: ดอกไม้, texture")
print("   ❌ ไม่ควรใช้กับ: ใบหน้า, สัตว์ (ดูแปลก)")

---

## ⬆️⬇️ Part 4: Shifting (การเลื่อน)

### ทำไมต้องเลื่อน?
- วัตถุอาจไม่อยู่ตรงกลาง
- ภาพอาจถ่ายเอียง

In [ ]:
# ทดลองเลื่อนในทิศทางต่างๆ
from PIL import ImageChops

def shift_image(img, x_shift, y_shift):
    """เลื่อนภาพ"""
    return ImageChops.offset(img, x_shift, y_shift)

shifts = [
    (0, 0, 'Original'),
    (50, 0, 'Shift Right →'),
    (-50, 0, 'Shift Left ←'),
    (0, 50, 'Shift Down ↓'),
    (0, -50, 'Shift Up ↑'),
    (30, 30, 'Shift Diagonal ↘️')
]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (x, y, title) in enumerate(shifts):
    shifted = shift_image(original_img, x, y)
    
    axes[idx].imshow(shifted)
    axes[idx].set_title(title, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("💡 Shifting:")
print("   - ใช้ width_shift_range=0.2 (เลื่อน 20% ของความกว้าง)")
print("   - ใช้ height_shift_range=0.2 (เลื่อน 20% ของความสูง)")
print("   - ช่วยให้โมเดลเรียนรู้ว่าวัตถุอาจอยู่ตำแหน่งไหนก็ได้")

---

## 🔍 Part 5: Zooming (การซูม)

### ทำไมต้องซูม?
- วัตถุอาจอยู่ไกล-ใกล้
- ขนาดของวัตถุในภาพไม่เท่ากัน

In [ ]:
# ทดลองซูมในระดับต่างๆ
zoom_levels = [0.5, 0.75, 1.0, 1.25, 1.5, 2.0]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, zoom in enumerate(zoom_levels):
    # คำนวณขนาดใหม่
    new_size = tuple(int(dim * zoom) for dim in original_img.size)
    
    if zoom < 1.0:
        # Zoom out (ย่อ)
        zoomed = original_img.resize(new_size, Image.Resampling.LANCZOS)
        # Paste บนภาพขาว
        result = Image.new('RGB', original_img.size, 'white')
        offset = ((original_img.size[0] - new_size[0]) // 2,
                 (original_img.size[1] - new_size[1]) // 2)
        result.paste(zoomed, offset)
    else:
        # Zoom in (ขยาย)
        zoomed = original_img.resize(new_size, Image.Resampling.LANCZOS)
        # Crop กลาง
        left = (new_size[0] - original_img.size[0]) // 2
        top = (new_size[1] - original_img.size[1]) // 2
        result = zoomed.crop((left, top, left + original_img.size[0], top + original_img.size[1]))
    
    axes[idx].imshow(result)
    axes[idx].set_title(f'Zoom: {zoom}x', fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("💡 Zooming:")
print("   - zoom_range=0.2 (ซูม 0.8x ถึง 1.2x)")
print("   - Zoom in: เห็นรายละเอียดมากขึ้น")
print("   - Zoom out: เห็นภาพรวม")

---

## ☀️ Part 6: Brightness (ความสว่าง)

### ทำไมต้องปรับความสว่าง?
- ภาพถ่ายในแสงต่างกัน
- เช้า/เย็น, ในร่ม/กลางแจ้ง

In [ ]:
# ทดลองปรับความสว่างในระดับต่างๆ
brightness_levels = [0.3, 0.5, 0.7, 1.0, 1.3, 1.7]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, factor in enumerate(brightness_levels):
    enhancer = ImageEnhance.Brightness(original_img)
    brightened = enhancer.enhance(factor)
    
    axes[idx].imshow(brightened)
    title = f'Brightness: {factor}x'
    if factor < 1.0:
        title += ' (Darker 🌙)'
    elif factor > 1.0:
        title += ' (Brighter ☀️)'
    else:
        title += ' (Original)'
    axes[idx].set_title(title, fontsize=11, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("💡 Brightness:")
print("   - brightness_range=[0.7, 1.3]")
print("   - ช่วยให้โมเดลทำงานได้ในสภาพแสงต่างๆ")

---

## 🎨 Part 7: ImageDataGenerator - ทำ Augmentation แบบอัตโนมัติ

### Keras มี Tool ที่ทำให้ Augmentation ง่ายมาก!

In [ ]:
# สร้าง ImageDataGenerator พร้อม augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=40,           # หมุน ±40 องศา
    width_shift_range=0.2,       # เลื่อนซ้าย-ขวา 20%
    height_shift_range=0.2,      # เลื่อนบน-ล่าง 20%
    shear_range=0.2,             # เอียง
    zoom_range=0.2,              # ซูม ±20%
    horizontal_flip=True,        # พลิกซ้าย-ขวา
    fill_mode='nearest'          # วิธีเติมพื้นที่ว่าง
)

print("✅ ImageDataGenerator created with augmentation!")
print("\nAugmentation Parameters:")
print("  • Rotation: ±40°")
print("  • Shift: ±20%")
print("  • Shear: 0.2")
print("  • Zoom: ±20%")
print("  • Horizontal Flip: Yes")

In [ ]:
# แปลงภาพเป็น array
img_array = img_to_array(original_img)
img_array = img_array.reshape((1,) + img_array.shape)  # (1, height, width, 3)

# สร้างภาพ augmented
print("🎨 Generating augmented images...\n")

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.ravel()

# ภาพต้นฉบับ
axes[0].imshow(original_img)
axes[0].set_title('ORIGINAL', fontsize=12, fontweight='bold', color='red')
axes[0].axis('off')

# สร้าง 11 ภาพ augmented
i = 1
for batch in datagen.flow(img_array, batch_size=1):
    augmented_img = array_to_img(batch[0])
    axes[i].imshow(augmented_img)
    axes[i].set_title(f'Augmented #{i}', fontsize=11, fontweight='bold')
    axes[i].axis('off')
    
    i += 1
    if i >= 12:  # 1 original + 11 augmented
        break

plt.tight_layout()
plt.show()

print("✨ จาก 1 ภาพ → สร้างได้หลายแบบ!")
print("   ทุกภาพแตกต่างกัน แต่ยังเป็นภาพเดียวกัน")
print("   โมเดลจะเห็นหลากหลายมุมมอง → เรียนรู้ดีขึ้น!")

---

## 📊 Part 8: เปรียบเทียบ Training ด้วย/ไม่มี Augmentation

มาดูกันว่า Augmentation มีผลยังไง!

In [ ]:
# จำลองผลการเทรน
epochs = np.arange(1, 51)

# ไม่มี Augmentation - Overfit!
train_acc_no_aug = 0.5 + 0.45 * (1 - np.exp(-epochs / 10))
val_acc_no_aug = 0.5 + 0.30 * (1 - np.exp(-epochs / 10)) - 0.1 * (epochs / 50)

# มี Augmentation - Better generalization!
train_acc_aug = 0.5 + 0.40 * (1 - np.exp(-epochs / 15))
val_acc_aug = 0.5 + 0.38 * (1 - np.exp(-epochs / 15))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# ไม่มี Augmentation
axes[0].plot(epochs, train_acc_no_aug, 'b-', linewidth=2, label='Training Accuracy')
axes[0].plot(epochs, val_acc_no_aug, 'r-', linewidth=2, label='Validation Accuracy')
axes[0].fill_between(epochs, train_acc_no_aug, val_acc_no_aug, alpha=0.3, color='orange')
axes[0].set_title('WITHOUT Data Augmentation\n❌ Overfitting!', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].text(25, 0.7, 'GAP = Overfitting', fontsize=12, bbox=dict(boxstyle='round', facecolor='red', alpha=0.5))

# มี Augmentation
axes[1].plot(epochs, train_acc_aug, 'b-', linewidth=2, label='Training Accuracy')
axes[1].plot(epochs, val_acc_aug, 'g-', linewidth=2, label='Validation Accuracy')
axes[1].fill_between(epochs, train_acc_aug, val_acc_aug, alpha=0.3, color='lightgreen')
axes[1].set_title('WITH Data Augmentation\n✅ Better Generalization!', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].text(25, 0.85, 'Small GAP = Good!', fontsize=12, bbox=dict(boxstyle='round', facecolor='green', alpha=0.5))

plt.tight_layout()
plt.show()

print("📊 Results:")
print("\n❌ WITHOUT Augmentation:")
print(f"   Training Acc: {train_acc_no_aug[-1]:.2%}")
print(f"   Validation Acc: {val_acc_no_aug[-1]:.2%}")
print(f"   Gap: {(train_acc_no_aug[-1] - val_acc_no_aug[-1]):.2%} (Overfitting!)")

print("\n✅ WITH Augmentation:")
print(f"   Training Acc: {train_acc_aug[-1]:.2%}")
print(f"   Validation Acc: {val_acc_aug[-1]:.2%}")
print(f"   Gap: {(train_acc_aug[-1] - val_acc_aug[-1]):.2%} (Much better!)")

---

## 📝 Part 9: Best Practices & Tips

### ✅ แนะนำให้ใช้:

In [ ]:
print("🎯 Recommended Augmentation Settings:\n")
print("="*70)

print("\n1️⃣ For General Images (photos, objects):")
print("""
ImageDataGenerator(
    rotation_range=20,           # หมุนไม่เยอะเกิน
    width_shift_range=0.2,       # เลื่อน 20%
    height_shift_range=0.2,
    zoom_range=0.2,              # ซูม ±20%
    horizontal_flip=True,        # พลิกซ้าย-ขวา OK
    fill_mode='nearest'
)
""")

print("\n2️⃣ For Medical Images (X-rays, MRI):")
print("""
ImageDataGenerator(
    rotation_range=10,           # หมุนน้อย (ทิศทางสำคัญ)
    width_shift_range=0.1,       # เลื่อนน้อย
    height_shift_range=0.1,
    zoom_range=0.1,              # ซูมเบาๆ
    horizontal_flip=False,       # ❌ อย่าพลิก (ซ้าย-ขวาต่างกัน)
    fill_mode='constant'
)
""")

print("\n3️⃣ For Text/Documents (OCR):")
print("""
ImageDataGenerator(
    rotation_range=5,            # หมุนน้อยมาก (ข้อความต้องตรง)
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=False,       # ❌ ห้ามพลิก (อ่านไม่ได้)
    fill_mode='constant'
)
""")

print("\n4️⃣ For Fashion/Clothing:")
print("""
ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,        # ✅ พลิกได้
    brightness_range=[0.8, 1.2], # ปรับแสง
    fill_mode='nearest'
)
""")

In [ ]:
print("\n⚠️ Common Mistakes to Avoid:\n")
print("="*70)

print("\n❌ DON'T:")
print("   • ใช้ rotation_range มากเกิน (>45°)")
print("   • พลิกภาพที่มีความหมายด้านซ้าย-ขวา (ตัวเลข, ข้อความ)")
print("   • Augment test data (ห้าม! test ต้องเป็นข้อมูลจริง)")
print("   • ใช้ augmentation มากเกินจนภาพเพี้ยน")

print("\n✅ DO:")
print("   • เริ่มจาก parameters น้อยๆ แล้วค่อยเพิ่ม")
print("   • ตรวจสอบภาพ augmented ก่อนเทรน (ดูว่าเพี้ยนไหม)")
print("   • Augment เฉพาะ training data")
print("   • ใช้ตาม domain ของข้อมูล (medical ≠ fashion)")

---

## 🎓 สรุปบทเรียน

### ✅ สิ่งที่เราเรียนรู้:

1. **ทำไมต้อง Augment?**
   - เพิ่มข้อมูล จาก 100 → 1000+ ภาพ
   - ป้องกัน Overfitting
   - โมเดลเห็นหลากหลายมุมมอง

2. **เทคนิค Augmentation:**
   - 🔄 Rotation (หมุน)
   - ↔️ Flip (พลิก)
   - ⬆️⬇️ Shift (เลื่อน)
   - 🔍 Zoom (ซูม)
   - ☀️ Brightness (ความสว่าง)

3. **ImageDataGenerator:**
   - ทำ Augmentation อัตโนมัติ
   - Real-time ตอน training
   - ไม่ต้องเก็บภาพเพิ่ม (ประหยัด storage)

4. **ผลลัพธ์:**
   - Validation accuracy สูงขึ้น
   - Gap ระหว่าง train-val ลดลง
   - โมเดล generalize ได้ดีกว่า

### 🚀 Next Step:

👉 [Train/Test Split & Data Leakage](05_train_test_split.ipynb)

เรียนรู้วิธีแบ่งข้อมูลอย่างถูกต้อง และหลีกเลี่ยง Data Leakage!